In [1]:
import cp_optimizer
import utils

In [11]:
q_list = ["q1", "q3", "q2", "q4","q2","q2", "q5", "q6"]


In [12]:
df_queries = utils.load(q_list, num_partitions='16')
res = utils.interpolate(df_queries)
precision = 10
C =16
Q = len(q_list)
R = int(round(Q / 2))

In [13]:
q_splitted, procees_time = cp_optimizer.split(q_list, res, C, precision)

In [14]:
q_splitted

[['q6', 'q4', 'q6', 'q3'],
 ['q3', 'q4', 'q2', 'q2', 'q2', 'q2', 'q2', 'q4'],
 ['q1', 'q2', 'q2', 'q5', 'q1', 'q3', 'q5', 'q1']]

In [15]:
results = []
for qq in q_splitted:
    results.append(cp_optimizer.optimize(qq, res, C, R, precision))
    print('done')

^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.
^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.
^C pressed 2 times. Interrupting the solver. Press 3 times to force termination.


In [16]:
process_time, runtime, res_schedule = cp_optimizer.combine_results(results)

In [17]:
process_time, runtime, res_schedule

(1713.6121678352356,
 101.2,
 [[('q4', 5), ('q3', 11)],
  [('q6', 8), ('q6', 8)],
  [('q2', 16)],
  [('q2', 4), ('q2', 4), ('q2', 4), ('q2', 4)],
  [('q3', 8), ('q4', 4), ('q4', 4)],
  [('q1', 16)],
  [('q1', 8), ('q1', 8)],
  [('q5', 8), ('q5', 8)],
  [('q2', 4), ('q2', 4), ('q3', 8)]])